In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(n_samples=1000,
                            factor=.4,
                            noise=0.05)

# Let us visualize the dataset
reds = y == 0
blues = y == 1
plt.figure()
plt.scatter(X[reds, 0], X[reds, 1],
            c="red", s=20, edgecolor='k')
plt.scatter(X[blues, 0], X[blues, 1],
            c="blue", s=20, edgecolor='k')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=64, input_dim=2, activation="relu"))
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adamW', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=5,
          verbose=1)

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test)

In [ ]:
from medmnist import BloodMNIST
dataset = BloodMNIST(split='train', download=True)

In [ ]:
dataset.info

In [ ]:
import os
import numpy as np
dataset_name = 'bloodmnist'
data = np.load(os.path.join(dataset.root, dataset_name + '.npz'))

X_train, y_train= data['train_images'],data['train_labels']
X_val, y_val    = data['val_images'],  data['val_labels']
X_test, y_test  = data['test_images'], data['test_labels']

In [ ]:
num_train, height, width, channels  = X_train.shape
print("There are", num_train,"training examples  ")
print("Each sample has a size of", height,"by", width,"pixels")

num_val  = X_val.shape[0]
num_test = X_test.shape[0]

In [ ]:
num_classes = np.unique(y_train).shape[0]

In [ ]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_val   /= 255.0
X_test  /= 255.0

In [ ]:
plt.figure(figsize=(16,12))
for i in range(24):
    plt.subplot(4,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i].reshape((height,width,channels)), cmap=plt.cm.binary)
    plt.xlabel(dataset.info['label'][str(y_train[i][0])][:20])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

# Define parameters
batch_size = 128
epochs = 20

# Create the model
model = Sequential()
model.add(Flatten(input_shape=(height, width, channels)))  # Only need input_shape here
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adamW',
              metrics=['accuracy'])

In [ ]:
history_mlp = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

In [ ]:
def plot_history(histories):
    fig, ax = plt.subplots(1, 2,figsize=(15, 7))
    for (history,name) in histories:
        ax[0].plot(history.history['accuracy'],'o-.',label  = name+' train')
        ax[0].plot(history.history['val_accuracy'],'x-',label  = name+' test')

        ax[0].set_title('model accuracy')
        ax[0].set_ylabel('accuracy')
        ax[0].set_xlabel('epoch')
        ax[0].legend(loc='upper left')

    for (history,name) in histories:
        ax[1].plot(history.history['loss'],'o-.',label  = name+' train')
        ax[1].plot(history.history['val_loss'],'x-',label  = name+' test')

        ax[1].set_title('model loss')
        ax[1].set_ylabel('loss')
        ax[1].set_xlabel('epoch')
        ax[1].legend(loc='upper left')
    plt.show()

plot_history([(history_mlp,'MLP')])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3),
                 activation = 'relu',
                 input_shape = (height,width,channels)))

model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(filters = 16, kernel_size = (3,3),
                 activation = 'relu'))
model.add(Flatten())
model.add(Dense(8,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adamW',
              metrics=['accuracy'])

In [ ]:
X_train = X_train.reshape(num_train,height,width,channels)
X_test = X_test.reshape(num_test,height,width,channels)

In [ ]:
X_train.shape

In [ ]:
history_cnn = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

In [ ]:
plot_history([(history_mlp,'MLP'),
              (history_cnn,'CNN')])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation = 'relu',
                 input_shape = (height,width,channels)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(8,activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adamW',
              metrics=['accuracy'])

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
history_deep_cnn = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])